In [2]:
! wget https://www.dropbox.com/s/gfq7j9fhfihv3fd/dataset_train.json -O dataset_train.json
! wget https://www.dropbox.com/s/lv6xa80h5524spj/dataset_test.json -O dataset_test.json
! wget https://www.dropbox.com/s/wgyetw38nav2rvd/dataset.zip
! unzip dataset.zip

--2019-04-24 11:02:13--  https://www.dropbox.com/s/gfq7j9fhfihv3fd/dataset_train.json
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.1, 2620:100:6031:1::a27d:5101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/gfq7j9fhfihv3fd/dataset_train.json [following]
--2019-04-24 11:02:14--  https://www.dropbox.com/s/raw/gfq7j9fhfihv3fd/dataset_train.json
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uccebdb5b44536a60e02ab716d0a.dl.dropboxusercontent.com/cd/0/inline/AfmEyQcFzRnaHbYTnHogJiW4sT9fNznpTqoThNWhftWGj22oLwKDMShrH_YA7jfEQ4BNWELrboni7_NiuFyk0D5z43R4GrWoypIwl0rFqs7PPocVTTaf_WT-ucux0Htl4j4/file# [following]
--2019-04-24 11:02:14--  https://uccebdb5b44536a60e02ab716d0a.dl.dropboxusercontent.com/cd/0/inline/AfmEyQcFzRnaHbYTnHogJiW4sT9fNznpTqoThNWhftWGj22oLwKDMShrH_YA7jfEQ4BNWELrboni7_NiuFyk0D5z43R4G

In [0]:
import os
import json
import numpy as np
import cv2
from PIL import Image
from matplotlib import pyplot as plt

In [0]:
with open('dataset_train.json') as f:
    dataset_train = json.load(f)

In [0]:
def show(img, figsize=(5,5)):
    return
    plt.figure(figsize=figsize)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

In [0]:
def trim_numbers(img, n_digits):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, gray = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    _, contours, hierarchy = cv2.findContours(gray,
                                                                    cv2.RETR_LIST,
                                                                     cv2.CHAIN_APPROX_SIMPLE)
    cnt = []
    for c in contours:
        area = cv2.contourArea(c)
        if area < 20: #面積が小さすぎるときは無視
            continue
        x, y, w, h = cv2.boundingRect(c)
        if w > 100 or h > 100: #幅か高さが大きすぎるときは無視
            continue
        cnt.append(c)
    
    cnt = sorted(cnt, key=lambda x: cv2.boundingRect(x)[-1])
    cnt.reverse()
    
    numbers = cnt[:n_digits]
    show(cv2.drawContours(img.copy(), numbers, -1, (0, 255, 0), 1))
    
    if len(numbers) != n_digits:
        return None
    return numbers

In [0]:

def get_rects(numbers):
    numbers = sorted(numbers, key=lambda x:cv2.boundingRect(x)[0])
    result = []
    
    for i, c in enumerate(numbers):
        x, y, w, h = cv2.boundingRect(c)
        if h > w:
            size = h
            limit = w
        else:
            size = w
            limit = h
        p1 = int((size - limit) / 2)
        p2 = int((size + limit) / 2)
        rect_img = np.full((size, size, 3), 255, np.uint8)
        if h < w:
            rect_img[p1:p2, :] = img[y:y+h, x:x+w]
        else:
            rect_img[:, p1:p2] = img[y:y+h, x:x+w]
            
        rect_img = cv2.resize(rect_img, (28, 28))
        result.append(rect_img)
        show(rect_img, (1, 1))
        
    return result

In [0]:
def append_dataset(X, y, rects, label):
    for r, n in zip(rects, str(label)):
        m = r.reshape((1,) + r.shape)
        X = np.append(X, m, axis=0)
        y = np.append(y, [int(n)], axis=0)
    return X, y

In [56]:
X_train = np.empty((0, 28, 28, 3))
y_train = np.empty((0))

for i in range(len(dataset_train)):
    filename = dataset_train[i]['file']
    label = dataset_train[i]['total']
    n_digits = len(str(label))
    print(filename, label, n_digits)
    img = cv2.imread(os.path.join('dataset/train', filename), 1)
    height, width, _ = img.shape
    img = img[height-130:height-30, 110:width-50]
    height, width, _ = img.shape
    show(img)
    
    numbers = trim_numbers(img, n_digits)
    if not numbers:
        continue

    rects = get_rects(numbers)

    X_train, y_train = append_dataset(X_train, y_train, rects, label)

np.save('X_train.npy', X_train)
np.save('y_train.npy', y_train)

0.png 891 3
1.png 2211 4
2.png 3579 4
3.png 1990 4
4.png 792 3
5.png 3487 4
6.png 1655 4
7.png 869 3
8.png 2786 4
9.png 3188 4
10.png 3093 4
11.png 1725 4
12.png 1329 4
13.png 895 3
14.png 2096 4
15.png 103 3
16.png 1092 4
17.png 1249 4
18.png 2449 4
19.png 1036 4
20.png 1748 4
21.png 448 3
22.png 3914 4
23.png 3374 4
24.png 112 3
25.png 796 3
26.png 700 3
27.png 110 3
28.png 208 3
29.png 575 3
30.png 116 3
31.png 1792 4
32.png 1160 4
33.png 2749 4
34.png 1258 4
35.png 416 3
36.png 2233 4
37.png 345 3
38.png 2216 4
39.png 2309 4
40.png 3614 4
41.png 396 3
42.png 1720 4
43.png 1925 4
44.png 728 3
45.png 3960 4
46.png 2694 4
47.png 908 3
48.png 2115 4
49.png 2427 4
50.png 1150 4
51.png 904 3
52.png 1773 4
53.png 1394 4
54.png 495 3
55.png 1941 4
56.png 1353 4
57.png 2927 4
58.png 2260 4
59.png 297 3
60.png 460 3
61.png 1713 4
62.png 1899 4
63.png 1536 4
64.png 1008 4
65.png 1900 4
66.png 1752 4
67.png 981 3
68.png 2293 4
69.png 3377 4
70.png 2662 4
71.png 4819 4
72.png 1705 4
73.png 110 